In [4]:
import pandas as pd
import re
import html
import nltk
#nltk.download() #only for the first time running it
from nltk.corpus import stopwords
from nltk import PorterStemmer
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json

In [ ]:
# counter = 0
# tweets = []
# referencedDict = { "id":[], "retweet_count":[], "quote_count":[], "reply_count":[], "like_count":[]}

# # tweetData["id"].loc[0]
# # tweetData["public_metrics"].loc[0]
import json
import pandas as pd
import regex as re
#create dataframe on media posts

def createDataFrame():  
    with open("datain/nft_search_tweets_sample.jsonl", encoding='utf8') as f:
        for line in f:
            row = json.loads(line)
            try:
                if row["lang"] == 'en' and len(row.get('referenced_tweets', [])) == 0:
                    yield (row['id'],
                            row['text'],
                            row['public_metrics']['retweet_count'],
                            row['public_metrics']['quote_count'],
                            row['public_metrics']['reply_count'],
                            row['public_metrics']['like_count'],
                            row['attachments']['media_keys'])
            except KeyError:
                pass

#create csv
df = pd.DataFrame(createDataFrame())
df.columns =['id', 'text', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'media_keys']
df.to_json('datain/nft_tweets.jsonl', orient='records', index=True, lines= True)
output = pd.read_json("datain/nft_tweets.jsonl", lines = True)
output['total'] = output[['retweet_count', 'quote_count','reply_count', 'like_count']].sum(axis=1)
#sort by highest total
output = output.sort_values(by = 'total', ascending = False)
#top 100
output = output.head(100)
output.to_json('datain/nft_top_100_tweets.jsonl', orient='records', index=True, lines= True)

In [6]:
#import the data
data = pd.read_json("datain/nft_top_100_tweets.jsonl", lines=True)
#data = pd.read_json("datain/nft_search_tweets_sample/nft_search_tweets_sample.jsonl", lines=True)
#data = data.iloc[0:100000]
data

,id,text,retweet_count,quote_count,reply_count,like_count,media_keys,total
0,1369983392155009024,Celebrating LilMoon Rockets #NFT Reveal Givewa...,917,186,1920,1076,[3_1369983389210710019],4099
1,1395797789662187520,Fleek is officially running on the #InternetCo...,820,22,57,2522,[3_1395783651007377411],3421
2,1393223147630075904,$SFT is now live and ready to trade on #Uniswa...,1545,5,5,865,[3_1393223052079546377],2420
3,1375754272437116928,Let’s knock NFTs on the head. I want...some re...,762,48,26,1326,[7_1375754125414166528],2162
4,1380255678925742080,New drop coming tomorrow :) Stay tuned!!! #NFT...,97,7,35,1836,[3_1380255253862379521],1975
...,...,...,...,...,...,...,...,...
95,1398432709287030784,💎 Genesis.json 💎\n\n✨ Artwork by @BT\n💰 Sold t...,27,6,8,126,[3_1398432708418805766],167
96,1394577401137569792,"You know I love you guys, so every week, start...",34,4,65,64,[16_1394577383311745028],167
97,1374012202508689408,NFTs are in the works for Funko. Will be inter...,9,3,54,100,[3_1374012197332918274],166
98,1376921740849209344,Woke up this morning to the WONDERFUL surprise...,20,13,20,112,"[3_1376921334437928960, 3_1376921406638682116]",165


In [8]:
#prep stop words
stop_words = stopwords.words('english')
stop_words.append('rt')
stop_words.append('nft')
# stop_words.append('#nft')

#function for cleaning a tweet (remove mentions, hashtags, links, html entities, stop words. And make sure it's only letters)
def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        tweet = str.lower(tweet)
        # tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
        tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)|(#[A-Za-z0-9_]+)", " ", tweet).split()) # remove mentions and hashtags
        # tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)", " ", tweet).split())
        tweet = re.sub("(http\S+|http)", "", tweet, flags=re.MULTILINE) # remove links
        tweet = re.sub('\&\w+', "", tweet) # remove html entities
        tweet = re.sub('[^a-zA-Z# ]+', ' ', tweet) # make sure tweet is only letters
        # stem & remove stop words
        # tweet = ' '.join([PorterStemmer().stem(word=word) for word in tweet.split() if word not in stop_words])
        tweet = ' '.join([word for word in tweet.split() if word not in stop_words])
        return tweet

#clean data
for i in data.index:
    text = data["text"][i]
    cleaned_text = clean_tweet(text)
    cleaned_text = html.unescape(cleaned_text)
    data["text"][i] = cleaned_text

ipykernel_launcher:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
data

,id,text,retweet_count,quote_count,reply_count,like_count,media_keys,total
0,1369983392155009024,celebrating lilmoon rockets reveal giveway wor...,917,186,1920,1076,[3_1369983389210710019],4099
1,1395797789662187520,fleek officially running allowing dapps run we...,820,22,57,2522,[3_1395783651007377411],3421
2,1393223147630075904,sft live ready trade mcap circ supply price ac...,1545,5,5,865,[3_1393223052079546377],2420
3,1375754272437116928,let knock nfts head want revenge please watch ...,762,48,26,1326,[7_1375754125414166528],2162
4,1380255678925742080,new drop coming tomorrow stay tuned,97,7,35,1836,[3_1380255253862379521],1975
...,...,...,...,...,...,...,...,...
95,1398432709287030784,genesis json artwork sold eth,27,6,8,126,[3_1398432708418805766],167
96,1394577401137569792,know love guys every week starting today givin...,34,4,65,64,[16_1394577383311745028],167
97,1374012202508689408,nfts works funko interesting see goes lines in...,9,3,54,100,[3_1374012197332918274],166
98,1376921740849209344,woke morning wonderful surprise anonymous ange...,20,13,20,112,"[3_1376921334437928960, 3_1376921406638682116]",165


In [8]:
#write the cleaned tweets to a text file
with open('cleaned.txt', 'w') as f:
    for index in data.index:
        text = data["text"][index]
        f.write(text)
        f.write('\n')

In [11]:
from textblob import TextBlob
import json
import pandas as pd
positivity = ''
def getSentiment():
    with open("datain/cleaned.txt", encoding='utf8') as f:
        for line in f:
            row = TextBlob(line)
            if row.sentiment.polarity >= 0.7:
                positivity = 'positive'
            else:
                positivity = 'negative'
            yield line, row.sentiment.polarity, row.sentiment.subjectivity, positivity
df = pd.DataFrame(getSentiment())
df.columns =['text', 'polarity', 'subjectivity', positivity]
# df.to_json('datain/nft_top_100_sentiment.jsonl', orient='records', index=True, lines= True)
total = 0
for i in df['polarity']:
    total += i
total/100

KeyboardInterrupt: 

In [28]:
from textblob import TextBlob
import json
import pandas as pd
positivity = ''
def getSentiment():
    with open("datain/cleaned.txt", encoding='utf8') as f:
        for line in f:
            row = TextBlob(line)
            if row.sentiment.polarity >= 0.7:
                positivity = 'mostly_positive'
            elif row.sentiment.polarity <= -0.7:
                positivity = 'mostly_negative'
            elif row.sentiment.polarity > -0.7 and row.sentiment.polarity < -0.4:
                positivity = 'negative'
            elif row.sentiment.polarity > 0.4 and row.sentiment.polarity < 0.7:
                 positivity = 'positive'
            else:
                positivity = 'nuetral'
            yield line, row.sentiment.polarity, row.sentiment.subjectivity, positivity

df = pd.DataFrame(getSentiment())
df.columns =['text', 'polarity', 'subjectivity', 'positivity']
data['polarity'] = df['polarity']
data['subjectivity'] = df['subjectivity']
data['positivity'] = df['positivity']
# df.to_json('datain/nft_top_100_sentiment.jsonl', orient='records', index=True, lines= True)
df2 = data.sort_values(by = 'polarity', ascending = True)
df2.to_json('datain/nft_top_100_polarity_ascending.jsonl', orient='records', index=True, lines= True)
data

,id,text,retweet_count,quote_count,reply_count,like_count,media_keys,total,polarity,subjectivity,positivity
0,1369983392155009024,celebrating lilmoon rockets reveal giveway wor...,917,186,1920,1076,[3_1369983389210710019],4099,0.311111,0.344444,nuetral
1,1395797789662187520,fleek officially running allowing dapps run we...,820,22,57,2522,[3_1395783651007377411],3421,0.083333,0.583333,nuetral
2,1393223147630075904,sft live ready trade mcap circ supply price ac...,1545,5,5,865,[3_1393223052079546377],2420,0.067677,0.533333,nuetral
3,1375754272437116928,let knock nfts head want revenge please watch ...,762,48,26,1326,[7_1375754125414166528],2162,0.212500,0.550000,nuetral
4,1380255678925742080,new drop coming tomorrow stay tuned,97,7,35,1836,[3_1380255253862379521],1975,0.136364,0.454545,nuetral
...,...,...,...,...,...,...,...,...,...,...,...
95,1398432709287030784,genesis json artwork sold eth,27,6,8,126,[3_1398432708418805766],167,0.000000,0.000000,nuetral
96,1394577401137569792,know love guys every week starting today givin...,34,4,65,64,[16_1394577383311745028],167,0.166667,0.233333,nuetral
97,1374012202508689408,nfts works funko interesting see goes lines in...,9,3,54,100,[3_1374012197332918274],166,0.383333,0.558333,nuetral
98,1376921740849209344,woke morning wonderful surprise anonymous ange...,20,13,20,112,"[3_1376921334437928960, 3_1376921406638682116]",165,0.333333,0.500000,nuetral


In [12]:
# Word cloud of top 100 NFT's


ValueError: Unexpected character found when decoding 'null'